# Deploying to AWS

This notebook contains scripts that we will deploy to AWS, starting with the original dataset already in an S3 bucket. The project itself is about detecting three different gaming behaviors using an XGBoost classifier. We will do data cleaning and run that process with AWS Glue, then do any data preprocessing left in a preprocessing script with Sagemaker. We will afterwards of course do model training in sagemaker, and start a pipeline there just for demonstration purposes.

First let's manage our imports and system paths.

In [1]:
%load_ext dotenv
%dotenv

import os
import sys
import boto3
from pathlib import Path

# Change to root directory
os.chdir('..')

# Create a folder for all our code
SRC_PATH = Path("src")
sys.path.extend([f"./{SRC_PATH}"])

# And we'll need our role's
glue_role = os.getenv('GLUE_ROLE')
sagemaker_role = os.getenv('SAGEMAKER_ROLE')
bucket = os.getenv('BUCKET')

## Glue

### ETL

In [2]:
(SRC_PATH / "etl").mkdir(parents=True, exist_ok=True)
sys.path.extend([f"./{SRC_PATH}/etl"])

In [3]:
%%writefile {SRC_PATH}/etl/script.py

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pandas as pd
from io import StringIO
import boto3

args = getResolvedOptions(sys.argv, ['JOB_NAME', 'INPUT_BUCKET', 'INPUT_KEY', 'OUTPUT_BUCKET', 'OUTPUT_KEY'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Read data from S3
s3_client = boto3.client('s3')
obj = s3_client.get_object(Bucket=args['INPUT_BUCKET'], Key=args['INPUT_KEY'])
df = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8')))

#target label encoding
df['EngagementLevel'] = df['EngagementLevel'].map({'Low': 0, 'Medium': 1, 'High': 2})

# Perform transformations to independent variables
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
df['GameDifficulty'] = df['GameDifficulty'].map({'Easy': 0, 'Medium': 1, 'Hard': 2})
df_encoded = pd.get_dummies(df, columns=['Location', 'GameGenre'], drop_first=True)

encoded_cols = list(set(df_encoded.columns) - set(df.columns))
df_encoded[encoded_cols] = df_encoded[encoded_cols].astype(int)

# Convert the DataFrame back to CSV
csv_buffer = StringIO()
df_encoded.to_csv(csv_buffer, index=False)

# Upload the transformed data to S3
s3_client.put_object(Bucket=args['OUTPUT_BUCKET'], Key=args['OUTPUT_KEY'], Body=csv_buffer.getvalue())

job.commit()

Overwriting src/etl/script.py


In [4]:
file_path = f"{(SRC_PATH / 'etl' / 'script.py').as_posix()}"
s3_client = boto3.client('s3')
bucket_name = 'gaming-behavior'
script_file_name = 'script.py'
s3_key = f'glue-scripts/{script_file_name}'

# Upload the script to S3
s3_client.upload_file(file_path, bucket_name, s3_key)
print(f'Script uploaded to s3://{bucket_name}/{s3_key}')

Script uploaded to s3://gaming-behavior/glue-scripts/script.py


In [5]:
glue_client = boto3.client('glue')

# Parameters for the Glue job
job_name = 'etl-job'
script_location = f's3://{bucket_name}/{s3_key}'

# S3 locations for input and output data
input_bucket = 'gaming-behavior'
input_key = 'raw_data/online_gaming_behavior_dataset.csv'
output_bucket = 'gaming-behavior'
output_key = 'transformed_data/transformed_online_gaming_behavior_dataset.csv'

# Create or update the Glue job
response = glue_client.create_job(
    Name=job_name,
    Role=glue_role,
    Command={
        'Name': 'glueetl',
        'ScriptLocation': script_location,
        'PythonVersion': '3'
    },
    DefaultArguments={
        '--job-language': 'python',
        '--enable-continuous-cloudwatch-log': 'true',
        '--enable-spark-ui': 'true',
        '--INPUT_BUCKET': input_bucket,
        '--INPUT_KEY': input_key,
        '--OUTPUT_BUCKET': output_bucket,
        '--OUTPUT_KEY': output_key
    },
    MaxRetries=0,
    MaxCapacity=2.0,
    Timeout=2880,
    GlueVersion='2.0'
)

print(f'Glue job {job_name} created successfully')


Glue job etl-job created successfully


In [6]:
start_response = glue_client.start_job_run(JobName=job_name)
print(f'Glue job {job_name} started successfully with run ID: {start_response["JobRunId"]}')

Glue job etl-job started successfully with run ID: jr_195ff9faba49f0f2b2324e1d2fa90e9672d0b305ba6b92941a48b8b49e9eb36d


## Sagemaker

### Pre-processing

In [41]:
import sagemaker
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.steps import CacheConfig

# don't redo steps if already done from previous failed jobs
cache_config = CacheConfig(enable_caching=True, expire_after="15d")

S3_LOCATION = f"s3://{bucket}"

In [8]:
f"{S3_LOCATION}/transformed_data"

's3://gaming-behavior/transformed_data'

In [9]:
sm_boto3 = boto3.client("sagemaker")
pipeline_session = PipelineSession(default_bucket=bucket)
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_session.region_name
print("Using bucket" + bucket)

Using bucketgaming-behavior


In [10]:
(SRC_PATH / "preprocessing").mkdir(parents=True, exist_ok=True)
sys.path.extend([f"./{SRC_PATH}/preprocessing"])

In [14]:
import pandas as pd

df = pd.read_csv('online_gaming_behavior_dataset.csv')

#target label encoding
df['EngagementLevel'] = df['EngagementLevel'].map({'Low': 0, 'Medium': 1, 'High': 2})

# Perform transformations to independent variables
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})
df['GameDifficulty'] = df['GameDifficulty'].map({'Easy': 0, 'Medium': 1, 'Hard': 2})
df_encoded = pd.get_dummies(df, columns=['Location', 'GameGenre'], drop_first=True)

encoded_cols = list(set(df_encoded.columns) - set(df.columns))
df_encoded[encoded_cols] = df_encoded[encoded_cols].astype(int)

In [15]:
from sklearn.model_selection import train_test_split

df_encoded = df_encoded.drop(columns=['PlayerID'])
df_train, df_test = train_test_split(df_encoded, test_size=0.2)

y_train = df_train.EngagementLevel
y_test = df_test.EngagementLevel

X_train = df_train.drop("EngagementLevel", axis=1)
X_test = df_test.drop("EngagementLevel", axis=1)

In [42]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [54]:
%%writefile {SRC_PATH}/preprocessing/script.py

from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



def preprocess(base_directory):
    """Load the supplied data, split it and transform it."""
    df = _read_data_from_input_csv_files(base_directory)

    # the only transformation we need to do is drop the player id and split the data
    # everything else was done in the etl script
    
    df.drop(columns=['PlayerID'])
    df_train, df_test = train_test_split(df, test_size=0.2)

    y_train = df_train.EngagementLevel
    y_test = df_test.EngagementLevel

    X_train = df_train.drop("EngagementLevel", axis=1)
    X_test = df_test.drop("EngagementLevel", axis=1)

    _save_splits(base_directory, X_train, y_train, X_test, y_test)


def _read_data_from_input_csv_files(base_directory):
    """Read the data from the input CSV files.

    This function reads every CSV file available and
    concatenates them into a single dataframe.
    """
    input_directory = Path(base_directory) / "input"
    files = list(input_directory.glob("*.csv"))

    if len(files) == 0:
        message = f"The are no CSV files in {input_directory.as_posix()}/"
        raise ValueError(message)

    raw_data = [pd.read_csv(file) for file in files]
    df = pd.concat(raw_data)

    # Shuffle the data
    return df.sample(frac=1, random_state=42)


def _save_splits(base_directory, X_train, y_train, X_test, y_test):
    """Save data splits to disk.

    This function concatenates the transformed features
    and the target variable, and saves each one of the split
    sets to disk.
    """
    train = pd.concat([X_train, y_train], axis=1)
    test = pd.concat([X_test, y_test], axis=1)

    train_path = Path(base_directory) / "train"
    test_path = Path(base_directory) / "test"

    train_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)

    pd.DataFrame(train).to_csv(train_path / "train.csv", header=True, index=False)
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=True, index=False)


if __name__ == "__main__":
    preprocess(base_directory="/opt/ml/processing")


Overwriting src/preprocessing/script.py


In [55]:
pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/transformed_data",
)

In [56]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(
    base_job_name="preprocess-data",
    framework_version="1.2-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=sagemaker_role,
    sagemaker_session=pipeline_session,
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [57]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

preprocessing_step = ProcessingStep(
    name="preprocess-data",
    step_args=processor.run(
        code=f"{(SRC_PATH / 'preprocessing' / 'script.py').as_posix()}",
        inputs=[
            ProcessingInput(
                source=dataset_location,
                destination="/opt/ml/processing/input",
            ),
        ],
        outputs=[
            ProcessingOutput(
                output_name="train",
                source="/opt/ml/processing/train",
                destination=f"{S3_LOCATION}/preprocessing/train",
            ),
            ProcessingOutput(
                output_name="test",
                source="/opt/ml/processing/test",
                destination=f"{S3_LOCATION}/preprocessing/test",
            )
        ],
    ),
    cache_config=cache_config
)

c:\Users\RaviB\anaconda3\envs\sagemaker_mini\lib\site-packages\sagemaker\workflow\pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [58]:
from sagemaker.workflow.pipeline import Pipeline

preprocessing_pipeline = Pipeline(
    name="preprocessing-pipeline-pipeline",
    parameters=[dataset_location],
    steps=[
        preprocessing_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=pipeline_session,
)

preprocessing_pipeline.upsert(role_arn=sagemaker_role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:590184030535:pipeline/preprocessing-pipeline-pipeline',
 'ResponseMetadata': {'RequestId': '7b4060b0-cc4a-4327-8cc2-68ba591581e4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7b4060b0-cc4a-4327-8cc2-68ba591581e4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '99',
   'date': 'Fri, 09 Aug 2024 00:47:37 GMT'},
  'RetryAttempts': 0}}

In [59]:
preprocessing_pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:590184030535:pipeline/preprocessing-pipeline-pipeline/execution/oo19ki6bsmg4', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x000001853A493AC0>)

### Modeling

In [49]:
(SRC_PATH / "modeling").mkdir(parents=True, exist_ok=True)
sys.path.extend([f"./{SRC_PATH}/modeling"])

In [71]:
%%writefile {SRC_PATH}/modeling/script.py

import argparse
import os
import json
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, cohen_kappa_score
from pathlib import Path
import joblib
import tarfile



def train(model_directory, train_path, test_path, learning_rate=0.1, max_depth=7,):

    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train = X_train.drop(X_train.columns[-1], axis=1)

    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test[X_test.columns[-1]]
    X_test = X_test.drop(X_test.columns[-1], axis=1)

    model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, eval_metric='mlogloss', learning_rate=learning_rate, max_depth=max_depth)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_test, y_pred)

    model_path = os.path.join(model_directory, "model.joblib")
    joblib.dump(model, model_path)

    with tarfile.open(os.path.join(model_directory, "model.tar.gz"), "w:gz") as tar:
        tar.add(model_path, arcname=os.path.basename(model_path))



if __name__ =='__main__':
    print("[INFO] Extracting arguements")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument('--learning_rate', type=float, default=0.1)
    parser.add_argument('--max_depth', type=int, default=7)

    args, _ = parser.parse_known_args()

    train(
        model_directory=os.environ["SM_MODEL_DIR"],
        train_path=os.environ["SM_CHANNEL_TRAIN"],
        test_path=os.environ["SM_CHANNEL_TEST"],
        learning_rate=args.learning_rate,
        max_depth=args.max_depth,
    )

Overwriting src/modeling/script.py


In [72]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.xgboost import XGBoost

estimator = XGBoost(
    entry_point="script.py",
    source_dir=f"{(SRC_PATH / 'modeling').as_posix()}",
    hyperparameters={
        "learning_rate": 0.1,
        "max_depth": 7,
    },
    framework_version="1.2-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
    instance_count=1,
    role=sagemaker_role,
    sagemaker_session=pipeline_session,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.xlarge.


In [73]:
def create_training_step(estimator):
    """Create a SageMaker TrainingStep using the provided estimator."""
    return TrainingStep(
        name="train-model",
        step_args=estimator.fit(
            inputs={
                "train": TrainingInput(
                    s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                        "train"
                    ].S3Output.S3Uri,
                    content_type="text/csv",
                ),
                "test": TrainingInput(
                    s3_data=preprocessing_step.properties.ProcessingOutputConfig.Outputs[
                        "test"
                    ].S3Output.S3Uri,
                    content_type="text/csv",
                )
            },
        ),
        cache_config=cache_config
    )

train_model_step = create_training_step(estimator)

In [74]:
from sagemaker.workflow.pipeline import Pipeline

train_pipeline = Pipeline(
    name="train-pipeline",
    parameters=[dataset_location],
    steps=[
        preprocessing_step,
        train_model_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=pipeline_session,
)

train_pipeline.upsert(role_arn=sagemaker_role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:590184030535:pipeline/train-pipeline',
 'ResponseMetadata': {'RequestId': 'd5227536-d976-4e69-bc4c-e50b5abb3f33',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd5227536-d976-4e69-bc4c-e50b5abb3f33',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '82',
   'date': 'Fri, 09 Aug 2024 01:32:16 GMT'},
  'RetryAttempts': 0}}

In [75]:
train_pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:590184030535:pipeline/train-pipeline/execution/dsnhjapm4ky5', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x000001853A493AC0>)

### Deploying Model

In [95]:
import boto3
import sagemaker
import pandas as pd
from io import StringIO
from sagemaker.serializers import CSVSerializer
from sagemaker.xgboost import XGBoostModel

In [81]:
model_data = f"{S3_LOCATION}/sagemaker-xgboost-dsnhjapm4ky5-rcmJhiITEj/output/model.tar.gz"

xgboost_model = XGBoostModel(
    model_data=model_data,
    role=sagemaker_role,
    framework_version="1.2-1",
    sagemaker_session=sagemaker.Session()
)

In [82]:
# Deploy the model to a SageMaker endpoint
predictor = xgboost_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge'
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.xlarge.
INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-08-09-01-52-17-541
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-08-09-01-52-18-763
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-08-09-01-52-18-763


------!

In [85]:
s3 = boto3.client('s3')

file_key = 'preprocessing/test/test.csv'

# Download the file content to a string
csv_obj = s3.get_object(Bucket=bucket, Key=file_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

# Use StringIO to create a file-like object
df = pd.read_csv(StringIO(csv_string))

df.head()

In [97]:
X_test = df.drop(columns='EngagementLevel')
sample = X_test.iloc[0].to_csv(header=False, index=False).strip()

In [98]:
predictor.serializer = CSVSerializer()

predictor.predict(sample)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (502) from primary with message "<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.14.0 (Ubuntu)</center>
</body>
</html>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-xgboost-2024-08-09-01-52-18-763 in account 590184030535 for more information.

In [ ]:
# Make a prediction
input_data = '6.3,3.3,6.0,2.5'  # Example input
response = predictor.predict(input_data)
print(response)

In [99]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: sagemaker-xgboost-2024-08-09-01-52-18-763
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-08-09-01-52-18-763
